# CamHD Video File to Azure Blob with Dask

## Upload file from URL to Microsoft Azure Blob Storage

In [ ]:
from azure.storage.blob import BlockBlobService, PublicAccess
from azure.storage.blob.models import Blob
import yaml

In [ ]:
import yaml
# load Azure storage account credentials
with open('/home/jovyan/.azure_credentials_ooitest.yaml', 'r') as f:
    credentials = yaml.load(f)
azure_storage_account_name = credentials['azure_storage_account_name']
azure_storage_account_key = credentials['azure_storage_account_key']

In [ ]:
block_blob_service = BlockBlobService(account_name='azure_storage_account_name', account_key='azure_storage_account_key')
container_name ='camhdtester'

In [ ]:

def run_sample():
       block_blob_service.copy_blob(container_name,'rawdata.oceanobservatories.org','https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/06/CAMHDA301-20190706T060000Z.log')



In [ ]:
# Main method.
if __name__ == '__main__':
    run_sample()

### Raw Server to Local Storage with delayed functions

In [ ]:
# we will use a recent mp4 for a smaller size from ooi's raw data server
import pandas as pd
## List of Inputs for Dask
urls = ["https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T050000.mp4",
        "https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T051500.mp4", 
        "https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T043000.mp4"]
link = 'https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T043000.mp4'
paths = '/home/jovyan/floc_data/'
local_filename = []
for i in urls:
    local_filename.append(i.split('/')[-1])


In [ ]:
#Trying dict comprehensions, because dataframes and lists are writing all downloads to the same file path... writing over the same file.
download_dict = dict(zip(urls, local_filename))


In [ ]:
# if you want to check your work
# print(download_dict)

#### Start a Dask cluster

In [ ]:
#get yer dask going
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=10)
cluster

In [ ]:
from dask.distributed import Client
import dask
client = Client(cluster)
client

In [ ]:
from dask import delayed

## Dask Dl Function

In [ ]:
#Here we will try to DL with Dask
### the print command did not work but I would like it to!
import requests
import os
import sys

@delayed
def delayed_dl_file(download_dict):
    ## we use k and v here for key and value from our dictionary (download_dict)
    for k,v in download_dict.items():
        # NOTE the stream=True parameter
        r = requests.get(k, stream=True)
        with open(os.path.join(paths, v), 'wb') as f:
            #Use chunks to download the file in "chunks" instead of loading the entire file at once.
            for chunk in r.iter_content(chunk_size=5024): 
               ## 200 is the code for HTTP status = "OK"
                if r.status_code == 200:
                    f.write(chunk)
        print("File" , k , "created in", paths)

In [ ]:
#set up your function to compute
delayed_dl = [] 
delayed_dl = delayed_dl_file(download_dict)

In [ ]:
%%time
dask.compute(delayed_dl)

# for multiple file downloads from the server without Dask

In [ ]:
# we will use a recent mp4 for a smaller size from ooi's raw data server
import pandas as pd
## List to dl
urls = ["https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T050000.mp4", "https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T051500.mp4", "https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T043000.mp4"]
link = 'https://rawdata.oceanobservatories.org/files/RS03ASHS/PN03B/06-CAMHDA301/2019/07/01/CAMHDA301-20190701T043000.mp4'
paths = '/home/jovyan/floc_data/'
local_filename = []
for i in urls:
    local_filename.append(i.split('/')[-1])
#Check your filename ;)
#print(filename)

In [ ]:
#Trying dict comprehensions, because dataframes and lists are writing all downloads to the same file path... writing over the same file.
download_dict = dict(zip(urls, local_filename))


In [ ]:
print(download_dict)

In [ ]:
# Here we create our download function without Dask
# Stream is used to download media files with requests. Use chunks to download the file in "chunks" instead of loading the entire file at once.
import requests
import os
import sys

def dl_file(download_dict):
    for k,v in download_dict.items():
        r = requests.get(k, stream=True)
        with open(os.path.join(paths, v), 'wb') as f:
            for chunk in r.iter_content(chunk_size=5024): 
                ## 200 is the code for HTTP status = "OK"
                if r.status_code == 200:
                    f.write(chunk)
    print("File" , filename , "created in", path)

In [ ]:
dl_file(download_dict)

 I wanted to do this with a dataframe and not a dictionary, but I failed

In [ ]:
# # make a dataframe to hold our information
# downloads = pd.DataFrame({'url': urls, 'filename': local_filename, 'path':paths}, index=[1,2,3])


In [ ]:
# # check that the files and urls match up
# pd.set_option('display.max_colwidth', -1)
# print(downloads)

### For Single File Raw Server to Local Storage 

In [ ]:
# def get_video(video):
#     try:
#         return r = requests.get(video, stream=True)
#         with open(os.path.join(path, filename), 'wb') as f:
#             for chunk in r.iter_content(chunk_size=5024): 
#                 if chunk: 
#                     f.write(chunk)
#     except:
#         return False

## It works! Now, if you wanna do some directory work look below


In [ ]:
# # This cell defines the function that will save our drive anon file in our data directory
# def run(): 
#     owd = os.getcwd()
#     #first change dir to path
#     os.chdir(path)
#     print("Now working in", os.getcwd(),)
#     #run download function to save video to data directory
#     download_file(url)
#     print("File" , filename , "created in", path)
#     #change dir back to original working directory (owd)
#     os.chdir(owd)
#     print("Now working back in", os.getcwd(),)

In [ ]:
# # Here we create our directory for floc data
# # ->>>>>>>> move this to a relative directory. 
# import os
# import sys

# for url in urls:
#     filename = url.split('/')[-1]   
#     if not os.path.exists(path+filename):
#         os.mkdir(path+filename)
#         print("Path", path+filename, "now paved")
#     else:    
#         print("Directory" , path+filename , "already exists")

# Blob It!!!!

In [ ]:
import yaml
# load Azure storage account credentials
with open('/home/jovyan/.azure_credentials_ooitest.yaml', 'r') as f:
    credentials = yaml.load(f)
azure_storage_account_name = credentials['azure_storage_account_name']
azure_storage_account_key = credentials['azure_storage_account_key']

In [ ]:
import azure.storage.blob  as ASB
import azure.storage.common

In [ ]:
blob_service = ASB.BlockBlobService(azure_storage_account_name, account_key=azure_storage_account_key)

#content_settings = ContentSettings(content_type = "video/mov")
#blob_service.create_blob_from_path("mycontainer","myblockblob","sunset.png",content_settings)

In [ ]:
source = '/home/jovyan/vent_two.mp4'

In [ ]:
blob_name = 'blobtest2'
container_name = 'movtest'
#blob_service.create_container(container_name);

In [ ]:
blob_service.create_blob_from_path(container_name, blob_name, source,)

In [ ]:
# delete all blobs in container
blob = blob_service.list_blobs(container_name)
#for a in blob:
       #blob_service.delete_blob(container_name = container_name, blob_name = a.name)

In [ ]:
dir(list(blob))

In [ ]:
ind_blob = list(blob)

In [ ]:
ind_blob.count()

In [ ]:
def append_block_from_url(container_name, blob_name, copy_source_url, timeout=20):
    

## Scrap

In [ ]:
# create a container for the velocity data
#from azure.storage.blob import BlockBlobService
blob_service = BlockBlobService(azure_storage_account_name, azure_storage_account_key)
container_name = 'movtest'
blob_service.create_container(container_name);

In [ ]:
azure.append_blob_from_path(container_name, blob_name, file_path, validate_content=False, maxsize_condition=None, progress_callback=None, lease_id=None, timeout=None)

In [ ]:
blob_service.create_file_from_bytes(share_name, directory_name, container_name, file, index=0, count=None, content_settings=None, metadata=None, validate_content=False, progress_callback=None, max_connections=2, timeout=None)